# Flan-T5 Reddit AITA Finetuning

## Configure Training

In [ ]:
%pip install transformers accelerate torch datasets peft bitsandbytes tensorboard py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# set to where you want to save model checkpoints and logs
%cd /content/drive/MyDrive/AITA_FINETUNED_MODELS

Mounted at /content/drive
/content/drive/MyDrive/AITA_FINETUNED_MODELS


In [ ]:
from huggingface_hub import login

login()

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, BitsAndBytesConfig, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from datasets import load_dataset

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "google/flan-t5-xl"

# The instruction dataset to use
dataset_name = "MattBoraske/reddit-AITA-submissions-and-comments-top-2k"

# Fine-tuned model name
new_model = "flan-t5-xl-reddit-AITA-top-2k"

# Flan-T5 fine-tuning encoder/decoder context windows
encoder_context_window = 1024
decoder_context_window = 256

# Load the entire model on the GPU 0
device_map = {"": 0}

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.05

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 16

# Batch size per GPU for evaluation
per_device_eval_batch_size = 16

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 5e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "adafactor"

# Learning rate schedule
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Log every X updates steps
logging_steps = 10

# Log first step
logging_first_step = True

# Logging strategy
logging_strategy = "steps"

# Save checkpoint every X updates steps
save_steps = 100

# Save strategy
save_strategy = "steps"

## Run Training

### Load PEFT Model

In [ ]:
def get_model_memory_size(model):
  total_size = 0
  for param in model.parameters():
      # param.nelement() gives the total number of elements in the tensor,
      # param.element_size() gives the size in bytes of each element in the tensor.
      total_size += param.nelement() * param.element_size()
  return total_size

def get_model_memory_size_gb(model):
  total_size_bytes = get_model_memory_size(model)
  total_size_gb = total_size_bytes / (1024 ** 3)  # Convert bytes to gigabytes
  return total_size_gb

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained( model_name)

# Load base model
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
  load_in_4bit=use_4bit,
  bnb_4bit_quant_type=bnb_4bit_quant_type,
  bnb_4bit_compute_dtype=compute_dtype,
  bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

model = AutoModelForSeq2SeqLM.from_pretrained(
  model_name,
  device_map=device_map,
  quantization_config=bnb_config,
  torch_dtype=torch.bfloat16
)

print(f"Base Model memory size: {get_model_memory_size_gb(model)} GB")

# Define LoRA Config
lora_config = LoraConfig(
 r=lora_r,
 lora_alpha=lora_alpha,
 target_modules=["q", "v"],
 lora_dropout=lora_dropout,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

# Add LoRA adapter
model = prepare_model_for_kbit_training(model)
peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()

print(f"PEFT Model memory size: {get_model_memory_size_gb(peft_model)} GB")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Your GPU supports bfloat16: accelerate training with bf16=True


...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Base Model memory size: 3.151836395263672 GB
trainable params: 37,748,736 || all params: 2,887,505,920 || trainable%: 1.3073128521932174
PEFT Model memory size: 3.5380477905273438 GB


### Tokenize Model Inputs

In [ ]:
def preprocess_function(sample):
  # Tokenize inputs
  model_inputs = tokenizer(sample['flanT5_instruction'], max_length=encoder_context_window, padding='max_length', truncation=True)

  # Tokenize targets with the `text_target` keyword argument
  labels = tokenizer(text_target=sample["top_comment_1"], max_length=decoder_context_window, padding='max_length', truncation=True)

  # Return model inputs - input_ids, attention_mask, labels
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

dataset = load_dataset(dataset_name)
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["submission_title", "submission_text", "submission_score", "submission_url", "submission_date", "top_comment_1", 'top_comment_2', 'top_comment_3', 'top_comment_4', 'top_comment_5', 'top_comment_6', 'top_comment_7', 'top_comment_8', 'top_comment_9', 'top_comment_10', 'top_comment_1_classification', 'top_comment_2_classification', 'top_comment_3_classification', 'top_comment_4_classification', 'top_comment_5_classification', 'top_comment_6_classification', 'top_comment_7_classification', 'top_comment_8_classification', 'top_comment_9_classification', 'top_comment_10_classification', 'ambiguity_score', 'flanT5_instruction', 'llama2_instruction'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Generating train split:   0%|          | 0/1600 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


### Train PEFT Model

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Create data collator
data_collator = DataCollatorForSeq2Seq(
  tokenizer,
  model=model,
  label_pad_token_id=0, # ignore tokenizer pad token in the loss
  pad_to_multiple_of=8
)

# Define training args
training_args = Seq2SeqTrainingArguments(
  output_dir=new_model,
  num_train_epochs=num_train_epochs,
  per_device_train_batch_size=per_device_train_batch_size,
  per_device_eval_batch_size=per_device_eval_batch_size,
  gradient_accumulation_steps=gradient_accumulation_steps,
  optim=optim,
  save_steps=save_steps,
  save_strategy=save_strategy,
  logging_steps=logging_steps,
  logging_first_step=logging_first_step,
  logging_strategy=logging_strategy,
  logging_dir=f"{new_model}/logs",
  learning_rate=learning_rate,
  weight_decay=weight_decay,
  fp16=fp16,
  bf16=bf16,
  max_grad_norm=max_grad_norm,
  max_steps=max_steps,
  warmup_ratio=warmup_ratio,
  group_by_length=group_by_length,
  lr_scheduler_type=lr_scheduler_type,
  report_to="tensorboard",
)

# Create trainer
trainer = Seq2SeqTrainer(
  model=peft_model,
  args=training_args,
  data_collator=data_collator,
  train_dataset=tokenized_dataset['train'],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Train model
trainer.train()
peft_model.push_to_hub(new_model) # ADD YOUR HF TOKEN IF NOT LOGGED IN ALREADY

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
1,45.426400
10,34.779500
20,9.821500
30,1.621700
40,0.841600
50,0.741400
60,0.712800
70,0.681400
80,0.715300
90,0.709800


adapter_model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MattBoraske/flan-t5-xl-reddit-AITA-top-2k/commit/443e70c47d824a2a5495b72251c9ddc886cb56fe', commit_message='Upload model', commit_description='', oid='443e70c47d824a2a5495b72251c9ddc886cb56fe', pr_url=None, pr_revision=None, pr_num=None)